前言:

在程序运行过程中，总会遇到各种各样的错误。

有的错误是程序编写有问题造成的，比如本来应该输出整数结果输出了字符串，这种错误我们通常称之为bug，bug是必须修复的。

有的错误是用户输入造成的，比如让用户输入email地址，结果得到一个空字符串，这种错误可以通过检查用户输入来做相应的处理。

还有一类错误是完全无法在程序运行过程中预测的，比如写入文件的时候，磁盘满了，写不进去了，或者从网络抓取数据，网络突然断掉了。这类错误也称为异常，在程序中通常是必须处理的，否则，程序会因为各种问题终止并退出。

Python内置了一套异常处理机制，来帮助我们进行错误处理。

此外，我们也需要跟踪程序的执行，查看变量的值是否正确，这个过程称为调试。Python的pdb可以让我们以单步方式执行代码。

最后，编写测试也很重要。有了良好的测试，就可以在程序修改后反复运行，确保程序输出符合我们编写的测试。


**1. 错误处理**

高级语言通常都内置了一套try...except...finally...的错误处理机制，Python也不例外。

**# Try**

In [ ]:
# 机制示例:
try:
    print('try...')
    r = 10 / 0
    print('result:', r)
except ZeroDivisionError as e:
    print('except:', e)
finally:
    print('finally...')
print('END')

# 当我们认为某些代码可能会出错时，就可以用try来运行这段代码
# 如果执行出错，则后续代码不会继续执行，而是直接跳转至错误处理代码，即except语句块
# 执行完except后，如果有finally语句块，则执行finally语句块，至此，执行完毕。
# 上面的代码在计算10 / 0时会产生一个除法运算错误：

try...
except: division by zero
finally...
END


In [ ]:
# 如果把除数0改成2，则执行结果如下:
try:
    print('try...')
    r = 10 / 2
    print('result:', r)
except ZeroDivisionError as e:
    print('except:', e)
finally:
    print('finally...')
print('END')

# 由于没有错误发生，所以except语句块不会被执行
# 但是finally如果有，则一定会被执行（可以没有finally语句）。

try...
result: 5.0
finally...
END


In [ ]:
# 可以有多个except来捕获不同类型的错误:
try:
    print('try...')
    r = 10 / int('a')
    print('result:', r)
except ValueError as e:
    print('ValueError:', e)
except ZeroDivisionError as e:
    print('ZeroDivisionError:', e)
else:
    print('no error!')
finally:
    print('finally...')
print('END')

# int()函数可能会抛出ValueError，所以我们用一个except捕获ValueError，用另一个except捕获ZeroDivisionError。
# 此外，如果没有错误发生，可以在except语句块后面加一个else，当没有错误发生时，会自动执行else语句。

try...
ValueError: invalid literal for int() with base 10: 'a'
finally...
END


*   Python的错误其实也是class，所有的错误类型都继承自BaseException，所以在使用except时需要注意的是，它不但捕获该类型的错误，还把其子类也“一网打尽”。
  ```
    try:
       foo()
    except ValueError as e:
       print('ValueError')
    except UnicodeError as e:
       print('UnicodeError')
  ```
*   第二个except永远也捕获不到UnicodeError，因为UnicodeError是ValueError的子类，如果有，也被第一个except给捕获了。

*   Python所有的错误都是从BaseException类派生的，常见的错误类型和继承关系看这里：

    https://docs.python.org/3/library/exceptions.html#exception-hierarchy



*   使用try...except捕获错误还有一个巨大的好处，就是可以**跨越多层调用**，比如函数main()调用bar()，bar()调用foo()，结果foo()出错了，这时，只要main()捕获到了，就可以处理：

In [ ]:
def foo(s):
    return 10 / int(s)

def bar(s):
    return foo(s) * 2

def main():
    try:
        bar('0')
    except Exception as e:
        print('Error:', e)
    finally:
        print('finally...')
# 也就是说，不需要在每个可能出错的地方去捕获错误，只要在合适的层次去捕获错误就可以了。
# 这样一来，就大大减少了写try...except...finally的麻烦。

**# 记录错误**



*   如果不捕获错误，自然可以让Python解释器来打印出错误堆栈，但程序也被结束了。既然我们能捕获错误，就可以把错误堆栈打印出来，然后分析错误原因，同时，让程序继续执行下去。

*   Python内置的logging模块可以非常容易地记录错误信息。通过配置，logging还可以把错误记录到日志文件里，方便事后排查。



In [ ]:
import logging

def foo(s):
    return 10 / int(s)

def bar(s):
    return foo(s) * 2

def main():
    try:
        bar('0')
    except Exception as e:
        logging.exception(e)

In [ ]:
main()
print('END')

ERROR:root:division by zero
Traceback (most recent call last):
  File "<ipython-input-7-8bafe1af53af>", line 11, in main
    bar('0')
  File "<ipython-input-7-8bafe1af53af>", line 7, in bar
    return foo(s) * 2
  File "<ipython-input-7-8bafe1af53af>", line 4, in foo
    return 10 / int(s)
ZeroDivisionError: division by zero


END


**# 抛出错误**



*   因为错误是class，捕获一个错误就是捕获到该class的一个实例。因此，错误并不是凭空产生的，而是有意创建并抛出的。Python的内置函数会抛出很多类型的错误，我们自己编写的函数也可以抛出错误
*   如果要抛出错误，首先根据需要，可以定义一个错误的class，选择好继承关系，然后，用raise语句抛出一个错误的实例。
*   只有在必要的时候才定义我们自己的错误类型。如果可以选择Python已有的内置的错误类型（比如ValueError，TypeError），尽量使用Python内置的错误类型。

In [ ]:
class FooError(ValueError):
    pass

def foo(s):
    n = int(s)
    if n==0:
        raise FooError('invalid value: %s' % s)
    return 10 / n

foo('0')
# 执行，可以最后跟踪到我们自己定义的错误



*   另一种处理错误的方式(如下所示代码): 在bar()函数中，我们明明已经捕获了错误，但是，打印一个ValueError!后，又把错误通过raise语句抛出去了。

*   这种错误处理相当常见。捕获错误目的只是记录一下，便于后续追踪。但是，由于当前函数不知道应该怎么处理该错误，所以，最恰当的方式是继续往上抛，让顶层调用者去处理。好比一个员工处理不了一个问题时，就把问题抛给他的老板，如果他的老板也处理不了，就一直往上抛，最终会抛给CEO去处理。

In [ ]:
def foo(s):
    n = int(s)
    if n==0:
        raise ValueError('invalid value: %s' % s)
    return 10 / n

def bar():
    try:
        foo('0')
    except ValueError as e:
        print('ValueError!')
        raise

bar()



*   raise语句如果不带参数，就会把当前错误原样抛出。
*   此外，在except中raise一个Error，还可以把一种类型的错误转化成另一种类型。只要是合理的转换逻辑就可以，但是，决不应该把一个IOError转换成毫不相干的ValueError。




In [ ]:
try:
    10 / 0
except ZeroDivisionError:
    raise ValueError('input error!')

**2. 调试**

程序能一次写完并正常运行的概率很小，基本不超过1%。总会有各种各样的bug需要修正。有的bug很简单，看看错误信息就知道，有的bug很复杂，我们需要知道出错时，哪些变量的值是正确的，哪些变量的值是错误的，因此，需要一整套调试程序的手段来修复bug。

**# print()**
*   第一种方法简单直接粗暴有效，就是用**print()**把可能有问题的变量打印出来看看。

In [ ]:
def foo(s):
    n = int(s)
    print('>>> n = %d' % n)
    return 10 / n

def main():
    foo('0')

main()
# 执行后在输出中查找打印的变量值
# >>> n = 0
# Traceback (most recent call last):
#  ...
# ZeroDivisionError: integer division or modulo by zero


**# assert**
*   用print()最大的坏处是将来还得删掉它，想想程序里到处都是print()，运行结果也会包含很多垃圾信息。所以，我们又有第二种方法。

*   **断言** —— 凡是用print()来辅助查看的地方，都可以用断言（assert）来替代：






In [ ]:
def foo(s):
    n = int(s)
    assert n != 0, 'n is zero!'
    return 10 / n

def main():
    foo('0')

# assert的意思是，表达式n != 0应该是True，否则，根据程序运行的逻辑，后面的代码肯定会出错。
# 如果断言失败，assert语句本身就会抛出AssertionError:
# Traceback (most recent call last):
#  ...
# AssertionError: n is zero!



*   程序中如果到处充斥着assert，和print()相比也好不到哪去。不过，启动Python解释器时可以用-O(字母O)参数来关闭assert。关闭后，你可以把所有的assert语句当成pass来看。




**# logging**
*   把print()替换为logging是第3种方式，和assert比，logging不会抛出错误，而且可以输出到文件:



In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
s = '0'
n = int(s)
logging.info('n = %d' % n)
print(10 / n)

# INFO:root:n = 0
# Traceback (most recent call last):
#  File "err.py", line 8, in <module>
#    print(10 / n)
# ZeroDivisionError: division by zero



*   这就是logging的好处，它允许你指定记录信息的级别，有debug，info，warning，error等几个级别，当我们指定level=INFO时，logging.debug就不起作用了。同理，指定level=WARNING后，debug和info就不起作用了。这样一来，你可以放心地输出不同级别的信息，也不用删除，最后统一控制输出哪个级别的信息。
*   logging的另一个好处是通过简单的配置，一条语句可以同时输出到不同的地方，比如console和文件。





**# pdb**
*   第4种方式是启动Python的调试器pdb，让程序以单步方式运行，可以随时查看运行状态。

**3. 单元测试**

如果你听说过“测试驱动开发”（TDD：Test-Driven Development），单元测试就不陌生。

单元测试是用来对一个模块、一个函数或者一个类来进行正确性检验的测试工作。

比如对函数abs()，我们可以编写出以下几个测试用例：

1. 输入正数，比如1、1.2、0.99，期待返回值与输入相同；

2. 输入负数，比如-1、-1.2、-0.99，期待返回值与输入相反；

3. 输入0，期待返回0；

4. 输入非数值类型，比如None、[]、{}，期待抛出TypeError。

把上面的测试用例放到一个测试模块里，就是一个完整的单元测试。

如果单元测试通过，说明我们测试的这个函数能够正常工作。如果单元测试不通过，要么函数有bug，要么测试条件输入不正确，总之，需要修复使单元测试能够通过。

单元测试通过后有什么意义呢？如果我们对abs()函数代码做了修改，只需要再跑一遍单元测试，如果通过，说明我们的修改不会对abs()函数原有的行为造成影响，如果测试不通过，说明我们的修改与原有行为不一致，要么修改代码，要么修改测试。

这种以测试为驱动的开发模式最大的好处就是确保一个程序模块的行为符合我们设计的测试用例。在将来修改的时候，可以极大程度地保证该模块行为仍然是正确的。

*   编写一个Dict类，这个类的行为和dict一致，但是可以通过属性来访问，用起来就像下面这样:



```
>>> d = Dict(a=1, b=2)
>>> d['a']
1
>>> d.a
1
```


In [2]:
# mydict.py代码如下:
class Dict(dict):

    def __init__(self, **kw):
        super().__init__(**kw)

    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Dict' object has no attribute '%s'" % key)

    def __setattr__(self, key, value):
        self[key] = value

In [4]:
d = Dict(a=1, b=2)
d['b']

2



*   编写单元测试时，我们需要编写一个测试类，从unittest.TestCase继承。
*   以test开头的方法就是测试方法，不以test开头的方法不被认为是测试方法，测试的时候不会被执行。


*   对每一类测试都需要编写一个test_xxx()方法。由于unittest.TestCase提供了很多内置的条件判断，我们只需要调用这些方法就可以断言输出是否是我们所期望的。最常用的断言就是assertEqual()：
    
    ```
    self.assertEqual(abs(-1), 1) # 断言函数返回的结果与1相等
    ```


*   另一种重要的断言就是期待抛出指定类型的Error，比如通过d['empty']访问不存在的key时，断言会抛出KeyError：
    ```
    with self.assertRaises(KeyError):
    value = d['empty']
    ```
*   而通过d.empty访问不存在的key时，我们期待抛出AttributeError：
    ```
    with self.assertRaises(AttributeError):
    value = d.empty
    ```








In [ ]:
# mydict_test.py代码如下:
import unittest

class TestDict(unittest.TestCase):

    def test_init(self):
        d = Dict(a=1, b='test')
        self.assertEqual(d.a, 1)
        self.assertEqual(d.b, 'test')
        self.assertTrue(isinstance(d, dict))

    def test_key(self):
        d = Dict()
        d['key'] = 'value'
        self.assertEqual(d.key, 'value')

    def test_attr(self):
        d = Dict()
        d.key = 'value'
        self.assertTrue('key' in d)
        self.assertEqual(d['key'], 'value')

    def test_keyerror(self):
        d = Dict()
        with self.assertRaises(KeyError):
            value = d['empty']

    def test_attrerror(self):
        d = Dict()
        with self.assertRaises(AttributeError):
            value = d.empty

**# 运行单元测试**
一旦编写好单元测试，我们就可以运行单元测试。最简单的运行方式是在mydict_test.py的最后加上两行代码:
```
if __name__ == '__main__':
    unittest.main()
```
这样就可以把mydict_test.py当做正常的python脚本运行:


```
$ python mydict_test.py
```

另一种方法是在命令行通过参数-m unittest直接运行单元测试:

```
$ python -m unittest mydict_test
.....
----------------------------------------------------------------------
Ran 5 tests in 0.000s

OK
```
这是推荐的做法，因为这样可以一次批量运行很多单元测试，并且，有很多工具可以自动来运行这些单元测试。

可以在单元测试中编写两个特殊的setUp()和tearDown()方法。这两个方法会分别在每调用一个测试方法的前后分别被执行。

setUp()和tearDown()方法有什么用呢？设想你的测试需要启动一个数据库，这时，就可以在setUp()方法中连接数据库，在tearDown()方法中关闭数据库，这样，不必在每个测试方法中重复相同的代码:



```
class TestDict(unittest.TestCase):

    def setUp(self):
        print('setUp...')

    def tearDown(self):
        print('tearDown...')
```








**4. 文档测试**

如果你经常阅读Python的官方文档，可以看到很多文档都有示例代码。比如re模块就带了很多示例代码：


```
>>> import re
>>> m = re.search('(?<=abc)def', 'abcdef')
>>> m.group(0)
'def'
```

可以把这些示例代码在Python的交互式环境下输入并执行，结果与文档中的示例代码显示的一致。

这些代码与其他说明可以写在注释中，然后，由一些工具来自动生成文档。既然这些代码本身就可以粘贴出来直接运行，那么，可不可以自动执行写在注释中的这些代码呢？

答案是肯定的。

当我们编写注释时，如果写上这样的注释：

```
def abs(n):
    '''
    Function to get absolute value of number.
    
    Example:
    
    >>> abs(1)
    1
    >>> abs(-1)
    1
    >>> abs(0)
    0
    '''
    return n if n >= 0 else (-n)
```
无疑更明确地告诉函数的调用者该函数的期望输入和输出。

并且，Python内置的“文档测试”（doctest）模块可以直接提取注释中的代码并执行测试。

doctest严格按照Python交互式命令行的输入和输出来判断测试结果是否正确。只有测试异常的时候，可以用...表示中间一大段烦人的输出。

让我们用doctest来测试上次编写的Dict类：



```
# mydict2.py
class Dict(dict):
    '''
    Simple dict but also support access as x.y style.

    >>> d1 = Dict()
    >>> d1['x'] = 100
    >>> d1.x
    100
    >>> d1.y = 200
    >>> d1['y']
    200
    >>> d2 = Dict(a=1, b=2, c='3')
    >>> d2.c
    '3'
    >>> d2['empty']
    Traceback (most recent call last):
        ...
    KeyError: 'empty'
    >>> d2.empty
    Traceback (most recent call last):
        ...
    AttributeError: 'Dict' object has no attribute 'empty'
    '''
    def __init__(self, **kw):
        super(Dict, self).__init__(**kw)

    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Dict' object has no attribute '%s'" % key)

    def __setattr__(self, key, value):
        self[key] = value

if __name__=='__main__':
    import doctest
    doctest.testmod()
```
运行python mydict2.py：

```
$ python mydict2.py
```

什么输出也没有。这说明我们编写的doctest运行都是正确的。如果程序有问题，比如把__getattr__()方法注释掉，再运行就会报错。



